In [1]:
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

import gzip
import json
import codecs
from multiprocessing.dummy import Pool, Queue

In [2]:
mas_of_urls=[]
for n in tqdm(range(1,14)):
    soup = BeautifulSoup(requests.get('https://gg.deals/games/?type=1&page='+str(n)).text, 'lxml')
    mas_of_candidats=soup.find_all('a', class_="main-image")
    for cand in mas_of_candidats:
        tek_url='https://gg.deals' + str(cand['href'])
        mas_of_urls.append(tek_url)
    
print(len(mas_of_urls))
mas_of_urls=mas_of_urls[0:300]
print(len(mas_of_urls))

100%|██████████| 13/13 [00:11<00:00,  1.09it/s]

312
300


In [14]:
def get_name(soup):
    otv=soup.find_all('span', itemprop='name')
    if otv is not None and len(otv)==4:
        otv=otv[3].text
    return otv

def get_img(soup):
    otv=soup.find('img',class_="image-game")
    if otv is not None:
        otv=otv['src']
    return otv

def get_market_url(soup):
    otv=soup.find_all('a',attrs={'class':'game-link-widget', 'href':True})
    if otv is not None and len(otv)>=1:
        otv=requests.get(otv[0]['href']).url 
    return otv


def get_wishlist_alert_owners_count(soup):
    otv=soup.find_all('span',class_="user-count")
    if otv is None:
        return None
    else:
        otv0=otv[0].find("span",class_='count')
        otv2=otv[2].find("span",class_='count')
        otv4=otv[4].find("span",class_='count')
        if otv0 is not None:
            otv0=otv0.text
        if otv2 is not None:
            otv2=otv2.text
        if otv4 is not None:
            otv4=otv4.text
        return otv0,otv2,otv4



def get_release_date(soup):
    otv=soup.find('div', class_="game-info-details-section game-info-details-section-release")
    if otv is not None:
        otv=otv.find("p", class_="game-info-details-content")
        if otv is not None:
            otv=otv.text
    return otv
                     
def get_developer(soup):
    otv=soup.find('div', class_="game-info-details-section game-info-details-section-developer")
    if otv is not None:
        otv=otv.find("p", class_="game-info-details-content")
        if otv is not None:
            otv=otv.text
    return otv



def get_metacritic_score(soup):
    otv=soup.find('a', class_="score-circle score-metascore")
    if otv is not None:
        otv=otv.find("span",class_="overlay")
        if otv is not None:
            otv=int(otv.text)
    return otv

def get_user_score(soup):
    otv=soup.find('a', class_="score-circle score-userscore")
    if otv is not None:
        otv=otv.find("span",class_="overlay")
        if otv is not None:
            otv=float(otv.text)
    return otv


def get_review_label_positive_pctg_count(soup):
    otv=soup.find('span', class_="reviews-label")
    if otv is None:
        return None
    else:
        otv1=re.sub(r'[\d (),]','',otv.text)
        otv2=int(otv['title'][0:2])
        otv3=otv.find("span",class_="semi-transparent")
        if otv3 is not None:
            otv3=int(re.sub(r'[,-.]', '',otv3.text.strip().strip("()")))
        return otv1, otv2, otv3



def get_genres(soup):
    otv=soup.find('div', id='game-info-genres')
    if otv is not None:
        otv=otv.find_all('a', class_='badge')
        if otv is not None:
            otv=[x.text for x in otv]
    return otv

def get_tags(soup):
    otv=soup.find('div', id='game-info-tags')
    if otv is not None:
        otv=otv.find_all('a', class_='badge')
        if otv is not None:
            otv=[x.text for x in otv]
    return otv

def get_features(soup):
    otv=soup.find('div', id='game-info-features')
    if otv is not None:
        otv=otv.find_all('a', class_='badge')
        if otv is not None:
            otv=[x.text for x in otv]
    return otv

def get_pc_systems(soup):
    otv=soup.find('div', class_='game-requirements-tabs')
    if otv is not None:
        otv=otv.find_all('a')
        otv=[x.text for x in otv]
    return otv

def get_dlcs(soup):
    otv=[]
    tmp=soup.find('section', id='game-dlcs')
    if tmp is not None:
        tmp=tmp.find_all('a', attrs={'class':'full-link', 'href':True})
        if tmp is not None:
            otv=[x['href'] for x in tmp]
    #print("len_otv=",len(otv))
    link_more=soup.find('div', class_='list-show-more')
    if link_more is not None:
        link_more=link_more.find('a',attrs={'data-url':True})
        if link_more is not None:
            link_more=link_more['data-url']
            if link_more is not None:
                headers={'x-requested-with':'XMLHttpRequest'}
                dop_soup=BeautifulSoup(requests.get('https://gg.deals' + link_more,headers=headers).text, 'lxml')
                dop_tmp=dop_soup.find_all('a', attrs={'class':'full-link', 'href':True})
                for x in dop_tmp:
                    otv.append(x['href'])
    
    if otv is not None and len(otv)==0:
        otv=None
    return otv

def get_pacs(soup):
    otv=[]
    tmp=soup.find('section', id='game-pacs')
    if tmp is not None:
        tmp=tmp.find_all('a', attrs={'class':'full-link', 'href':True})
        if tmp is not None:
            otv=[x['href'] for x in tmp]
    
    if otv is not None and len(otv)==0:
        otv=None
    return otv

def get_price_history(soup):
    adress=soup.find('div', class_='chart-container')
    if adress is None:
        return None
    if adress is not None:
        adress=adress['data-with-keyshops-url']
        headers={'x-requested-with':'XMLHttpRequest'}
        page=requests.get('https://gg.deals'+adress,headers=headers)
        file= BeautifulSoup(page.text, 'lxml')
        my_dict=json.loads(file.text)['chartData']['deals']
        price_history=[]
        for el in my_dict:
            tek={"ts":el['x']/1000, "price":el['y'], "shop":el['shop']}
            price_history.append(tek)
        return price_history

In [15]:
def process_page(url):
    soup=BeautifulSoup(requests.get(url).text, 'lxml')
    my_dict={}
    
    my_dict['url']=url
    
    tek_field=get_name(soup)
    if tek_field is not None:
        my_dict['name']=tek_field
        
    tek_field=get_img(soup)
    if tek_field is not None:
        my_dict['img']=tek_field
    tek_field=get_market_url(soup)
    if tek_field is not None:
        my_dict['market_url']=tek_field
        
    
    tek_field=get_wishlist_alert_owners_count(soup)
    if tek_field is not None:
        if tek_field[0] is not None:
            my_dict['wishlist_count']=tek_field[0]
        if tek_field[1] is not None:
            my_dict['alert_count']=tek_field[1]
        if tek_field[2] is not None:
            my_dict['owners_count']=tek_field[2]
  

    tek_field=get_release_date(soup)
    if tek_field is not None:
        my_dict['release_date']=tek_field
    tek_field=get_developer(soup)
    if tek_field is not None:
        my_dict['developer']=tek_field
        
        
    tek_field=get_metacritic_score(soup)
    if tek_field is not None:
        my_dict['metacritic_score']=tek_field
    tek_field=get_user_score(soup)
    if tek_field is not None:
        my_dict['user_score']=tek_field
        
        
    tek_field=get_review_label_positive_pctg_count(soup)
    if tek_field is not None:
        if tek_field[0] is not None:
            my_dict['review_label']=tek_field[0]
        if tek_field[1] is not None:
            my_dict['review_positive_pctg']=tek_field[1]
        if tek_field[2] is not None:
            my_dict['review_count']=tek_field[2]

    
   
    tek_field=get_genres(soup)
    if tek_field is not None:
        my_dict['genres']=tek_field
    tek_field=get_tags(soup)
    if tek_field is not None:
        my_dict['tags']=tek_field
    tek_field=get_features(soup)
    if tek_field is not None:
        my_dict['features']=tek_field
    
    tek_field=get_pc_systems(soup)
    if tek_field is not None:
        my_dict['pc_systems']=tek_field
        
    tek_field=get_dlcs(soup)
    if tek_field is not None:
        my_dict['dlcs']=tek_field
    tek_field=get_pacs(soup)
    if tek_field is not None:
        my_dict['pacs']=tek_field
    
    
    tek_field=get_price_history(soup)
    if tek_field is not None:
        my_dict['price_history']=tek_field
    
    return my_dict

In [16]:
%time test_parse=process_page('https://gg.deals/game/tricky-towers/')
test_parse

CPU times: user 389 ms, sys: 12.3 ms, total: 401 ms
Wall time: 2.14 s


{'url': 'https://gg.deals/game/tricky-towers/',
 'name': 'Tricky Towers',
 'img': 'https://img.gg.deals/70/ae/26c780669e69d3cd65f6bcd9f5aefa6dfaa6_307xt176.jpg',
 'market_url': 'https://store.steampowered.com/app/437920/',
 'wishlist_count': '527',
 'alert_count': '46',
 'owners_count': '3900',
 'release_date': '02 Aug 2016',
 'developer': 'WeirdBeard',
 'metacritic_score': 80,
 'user_score': 6.8,
 'review_label': 'VeryPositive',
 'review_positive_pctg': 90,
 'review_count': 6661,
 'genres': ['Casual', 'Indie'],
 'tags': ['Party Game',
  'Casual',
  'Puzzle',
  'Physics',
  'Building',
  'Funny',
  'Family Friendly',
  'Multiplayer',
  'Cartoony',
  'Arcade',
  'Singleplayer',
  'Online Co-Op',
  'Local Multiplayer',
  'Local Co-Op',
  '4 Player Local',
  'PvP',
  'Strategy',
  'Party',
  'Colorful',
  'Stylized',
  'Indie'],
 'features': ['Single-player',
  'Online PvP',
  'Shared/Split Screen PvP',
  'Steam Achievements',
  'Full controller support',
  'Steam Trading Cards',
  'Steam

In [20]:
queue = Queue()
for el in mas_of_urls:
    queue.put(el)  # очередь ссылок на книги


def process_page_wrapper(nomer_potoka):
    #print("Hello\n",queue.qsize())
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(nomer_potoka), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                record = process_page(queue.get())
                #print(record)
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
            except(ConnectionError):
                print("Exception caught, so moved to the next url;")

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 300/300 [02:11<00:00,  2.28it/s]


In [97]:
for url in tqdm(mas_of_urls):
    soup=BeautifulSoup(requests.get(url).text, 'lxml')
    otv=soup.find('div',class_='load-more-content').find('div', attrs={"data-container-game-id":True})
    if (otv=="61220"):
        print(otv['data-container-game-id'],url)

100%|██████████| 300/300 [02:27<00:00,  2.13it/s]


In [89]:
s= BeautifulSoup(requests.get('https://gg.deals/game/tricky-towers/').text, 'lxml')
s.find('div', class_='list-show-more').find('a',attrs={'data-url':True})

<a class="widget-link-more news-list-show-more-link load-more-button" data-url="/ru/games/relations/21937/?type=dlc&amp;offset=4&amp;hideKeyshops=0" href="#">Show more</a>

In [121]:

s= BeautifulSoup(requests.get('https://gg.deals/game/nier-replicant-ver122474487139/').text, 'lxml')
s.find('div', id_="official-stores")